In [1]:
## cremao_permit

# module
import psycopg2
import pandas as pd
import pymysql
import numpy as np
from shapely.wkb import loads

In [2]:
## 1.Setting

# 기준월
date = '202211'

# 광역시도코드
lawd_cd = ['11','26','27','28','29','30','31','36','41','42','43','44','45','46','47','48','50']

# Table 정보
dw_table = 'cremao_permit_point'

# ncp_table = 'permit_point'
columns = ['mgm_bldrgst_pk','pnu','point']

In [ ]:
## 2. M2 TABLE 생성

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# DELETE
aws_cursor.execute("delete from m2." + dw_table + " where 1=1;")
aws_conn.commit()

# M2.CREMAO_PERMIT INSERT
for i in lawd_cd:
    aws_cursor.execute(
    """
    insert into m2.""" + dw_table + """ (
    with date as (
    select '""" + date + """' as base_ym
    )
    select t1.mgm_bldrgst_pk -- PK
        , case when t1.pnu isnull then t2.pnu else t1.pnu end as pnu -- pnu 
        , t3.poly as point -- point
    from (
        select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu, *
        from m1.building_permit_info
        where arch_gb_cd_nm in ('신축','증축','개축','대수선','용도변경','이전') -- 허가구분 조건
        and arch_pms_day >= 2020-01-01 -- 허가일 기준 3년 데이터
        and base_ym = (select base_ym from date) -- 기준월
    ) as t1
    left join (
        select sigungu_cd||bjdong_cd||case when plat_gb_cd = '0' then '1' else plat_gb_cd end||bun||ji as pnu, *
        from m1.building_permit_flr_info
        where base_ym = (select base_ym from date) -- 기준월
    ) as t2
        on t1.mgm_bldrgst_pk = t2.mgm_bldrgst_pk
    left join (
        select pnu
             , case when right(jibun_jimok, 1) in ('전','답','과','목','임','광','염','대','장','학','차','주','창','도','철','제','천','구','유','양','수','공','체','원','종','사','묘','잡') 
                    then right(jibun_jimok, 1) end as jimok
             , ST_Transform(ST_SetSRID(poly::geometry, 5174), 4326)::geography as poly
        from m1.pnu
        where 1=1
            and substring(pnu, 1, 2) = '""" + i + """' -- 시도 
    ) t3
        on nvl(t1.pnu, t2.pnu) = t3.pnu
    left join (
        select pnu
             , listagg(prpos_area_dstrc_nm, ',') within group (order by prpos_area_dstrc_cd) as prpos_area_dstrc_nm
        from (
            select distinct pnu
                          , prpos_area_dstrc_cd
                          , prpos_area_dstrc_nm
            from m1.land_use
            where 1=1
                and substring(pnu, 1, 2) = '""" + i + """' -- 시도 
                and prpos_area_dstrc_cd in ('UQA111','UQA112','UQA121','UQA122','UQA123',
                                            'UQA130','UQA210','UQA220','UQA230','UQA240',
                                            'UQA310','UQA320','UQA330','UQA410','UQA420',
                                            'UQA430','UQB300','UQB200','UQB100','UQC001','UQD001')
        )
        group by 1
    ) t4
        on nvl(t1.pnu, t2.pnu, t3.pnu) = t4.pnu
    where t1.pnu notnull and t2.pnu notnull
    );
    """
    )
    aws_conn.commit()
    print(i, 'Done')

11 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
36 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
50 Done


In [3]:
## 3. NCP 이관

# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup) for tup in tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# insert
for i in lawd_cd:
    sql = "select * from m2." + dw_table + " where base_ym = '" + str(date) + "' and substring(pnu, 1, 2) = '" + str(i) + "' order by pnu;"
    aws_cursor.execute(sql)
    aws_conn.commit()
    rows = aws_cursor.fetchall()
    
    # dataframe
    df = pd.DataFrame(rows1, columns = columns)
    
    # null
    df = df.replace({np.nan: None})
    
    # val
    val = '(' + ('%s,' * len(df.columns))[:-1] + ')'
    
    # NCP insert
    for j in range(0, len(df), 1000):
    
        # DB Connect
        while True:
            try:
                conn = pymysql.connect(
                    host='db-bo6fn.pub-cdb.ntruss.com',
                    user='cremao',
                    password='cremao1234!!',
                    db='cremao',
                    charset='utf8'
                )
                break
            except Exception as err:
                pass

        # 1000개 단위로 나누기
        tmp = df[j:j + 1000]

        # insert
        execute_mogrify(conn, tmp, ncp_table, val)
        tmp = pd.DataFrame()

    # row count
    print(i, len(df))

11 911991
26 719310
27 602659
28 671673
29 391203
30 292897
31 504159
36 203320
41 5118995
42 2698190
43 2371001
44 3711711
45 3854192
46 5860288
47 5842377
48 4811532
50 873836
